In [3]:
# Instalar as bibliotecas necessárias
!pip install torch torchvision


In [4]:
# Conexão com o Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from google.colab.patches import cv2_imshow
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch.optim as optim
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Paths to your XML annotations and video file
xml_file = '/content/drive/MyDrive/approach_nwot/xml/annotations_new.xml'
video_file = '/content/drive/MyDrive/approach_nwot/videos/video_colorido_concatenado_10fps.mp4'

# Define crop intervals
color_crop_rows = (0, 720)
color_crop_cols = (250, 500)

def parse_annotations(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    annotations = {}

    for image in root.findall('image'):
        image_id = int(image.get('id'))
        image_name = image.get('name')

        boxes = []
        polygons = []
        ellipses = []

        for box in image.findall('box'):
            label = box.get('label')
            xtl = float(box.get('xtl'))
            ytl = float(box.get('ytl'))
            xbr = float(box.get('xbr'))
            ybr = float(box.get('ybr'))
            boxes.append({'label': label, 'xtl': xtl, 'ytl': ytl, 'xbr': xbr, 'ybr': ybr})

        for polygon in image.findall('polygon'):
            label = polygon.get('label')
            points = polygon.get('points')
            point_list = []
            for point in points.split(';'):
                x_str, y_str = point.split(',')
                x = float(x_str)
                y = float(y_str)
                point_list.append((x, y))
            polygons.append({'label': label, 'points': point_list})

        for ellipse in image.findall('ellipse'):
            label = ellipse.get('label')
            cx = float(ellipse.get('cx'))
            cy = float(ellipse.get('cy'))
            rx = float(ellipse.get('rx'))
            ry = float(ellipse.get('ry'))
            rotation = float(ellipse.get('rotation', '0.0'))  # Use default value if missing
            ellipses.append({'label': label, 'cx': cx, 'cy': cy, 'rx': rx, 'ry': ry, 'rotation': rotation})

        annotations[image_id] = {
            'name': image_name,
            'boxes': boxes,
            'polygons': polygons,
            'ellipses': ellipses
        }
    return annotations

def adjust_annotations(annotations, crop_rows, crop_cols):
    adjusted_annotations = {}
    for image_id, data in annotations.items():
        adjusted_boxes = []
        adjusted_polygons = []
        adjusted_ellipses = []

        for box in data['boxes']:
            xtl = box['xtl'] - crop_cols[0]
            ytl = box['ytl'] - crop_rows[0]
            xbr = box['xbr'] - crop_cols[0]
            ybr = box['ybr'] - crop_rows[0]
            # Check if the box is within the crop
            if (xtl >= 0 and ytl >= 0 and xbr <= crop_cols[1] - crop_cols[0] and ybr <= crop_rows[1] - crop_rows[0]):
                adjusted_boxes.append({
                    'label': box['label'],
                    'xtl': xtl,
                    'ytl': ytl,
                    'xbr': xbr,
                    'ybr': ybr
                })
        for polygon in data['polygons']:
            adjusted_points = []
            for x, y in polygon['points']:
                x_adj = x - crop_cols[0]
                y_adj = y - crop_rows[0]
                # Check if the point is within the crop
                if (x_adj >= 0 and x_adj <= crop_cols[1] - crop_cols[0] and y_adj >= 0 and y_adj <= crop_rows[1] - crop_rows[0]):
                    adjusted_points.append((x_adj, y_adj))
            if adjusted_points:
                adjusted_polygons.append({
                    'label': polygon['label'],
                    'points': adjusted_points
                })
        for ellipse in data['ellipses']:
            cx = ellipse['cx'] - crop_cols[0]
            cy = ellipse['cy'] - crop_rows[0]
            # Check if the center is within the crop
            if (cx >= 0 and cx <= crop_cols[1] - crop_cols[0] and cy >= 0 and cy <= crop_rows[1] - crop_rows[0]):
                adjusted_ellipses.append({
                    'label': ellipse['label'],
                    'cx': cx,
                    'cy': cy,
                    'rx': ellipse['rx'],
                    'ry': ellipse['ry'],
                    'rotation': ellipse['rotation']
                })

        adjusted_annotations[image_id] = {
            'name': data['name'],
            'boxes': adjusted_boxes,
            'polygons': adjusted_polygons,
            'ellipses': adjusted_ellipses
        }
    return adjusted_annotations

# Parse and adjust annotations
annotations = parse_annotations(xml_file)
adjusted_annotations = adjust_annotations(annotations, color_crop_rows, color_crop_cols)

# Prepare frames and targets
cap = cv2.VideoCapture(video_file)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("Total number of frames:", frame_count)

frames = []
targets = []

for frame_id in range(frame_count):
    ret, frame = cap.read()
    if not ret:
        break

    # Crop the frame
    cropped_frame = frame[color_crop_rows[0]:color_crop_rows[1], color_crop_cols[0]:color_crop_cols[1]]

    # Get the adjusted annotations for this frame
    if frame_id in adjusted_annotations:
        annots = adjusted_annotations[frame_id]

        # Collect the annotations
        boxes = []
        labels = []

        for box in annots['boxes']:
            xtl = int(box['xtl'])
            ytl = int(box['ytl'])
            xbr = int(box['xbr'])
            ybr = int(box['ybr'])
            boxes.append([xtl, ytl, xbr, ybr])
            labels.append(box['label'])
        # For simplicity, focusing on boxes
        if boxes:
            frames.append(cropped_frame)
            targets.append({'boxes': boxes, 'labels': labels})

    else:
        # If no annotations, skip
        continue

cap.release()

# Map labels to integers
label_set = set()
for target in targets:
    label_set.update(target['labels'])
label_map = {label: idx+1 for idx, label in enumerate(sorted(label_set))}  # +1 because 0 is background

# Update targets with integer labels
for target in targets:
    target['labels'] = [label_map[label] for label in target['labels']]

# Define the Dataset class
class CowEyeDataset(Dataset):
    def __init__(self, frames, targets, transforms=None):
        self.frames = frames
        self.targets = targets
        self.transforms = transforms

    def __getitem__(self, idx):
        img = self.frames[idx]
        target = self.targets[idx]

        # Convert image to PIL Image
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = torchvision.transforms.ToPILImage()(img)

        # Convert boxes and labels to tensors
        boxes = torch.as_tensor(target['boxes'], dtype=torch.float32)
        labels = torch.as_tensor(target['labels'], dtype=torch.int64)

        # Additional fields required by some models
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)

        # Create target dictionary
        target_dict = {}
        target_dict['boxes'] = boxes
        target_dict['labels'] = labels
        target_dict['image_id'] = image_id
        target_dict['area'] = area
        target_dict['iscrowd'] = iscrowd

        # Apply transforms if any
        if self.transforms:
            img = self.transforms(img)

        return img, target_dict

    def __len__(self):
        return len(self.frames)

# Create the dataset and dataloaders
dataset = CowEyeDataset(frames, targets, transforms=torchvision.transforms.ToTensor())
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Define the model
num_classes = len(label_map) + 1  # +1 for background
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)



Total number of frames: 81992


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 165MB/s]


In [6]:
# Contar o número total de parâmetros
total_params = sum(p.numel() for p in model.parameters())

# Contar o número de parâmetros treináveis
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total de parâmetros no modelo: {total_params}")
print(f"Total de parâmetros treináveis: {trainable_params}")


Total de parâmetros no modelo: 41299161
Total de parâmetros treináveis: 41076761


In [19]:
# Training setup
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
num_epochs = 100

# Training loop
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, targets in tqdm(train_loader):
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        epoch_loss += losses.item()

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {epoch_loss/len(train_loader)}")




100%|██████████| 20/20 [00:02<00:00,  8.42it/s]


Epoch 1, Loss: 0.012589297979138792


100%|██████████| 20/20 [00:02<00:00,  8.41it/s]


Epoch 2, Loss: 0.009168655378744006


100%|██████████| 20/20 [00:02<00:00,  8.38it/s]


...

Epoch 98, Loss: 0.0049946453073062


100%|██████████| 20/20 [00:02<00:00,  8.20it/s]


Epoch 99, Loss: 0.00519564087735489


100%|██████████| 20/20 [00:02<00:00,  8.38it/s]

Epoch 100, Loss: 0.005751325795426965


In [22]:
# Especifique o intervalo de frames que deseja analisar
inicio = 1600  # Frame inicial
fim = 2500     # Frame final

# Carregar o vídeo
cap = cv2.VideoCapture(video_file)

# Loop pelos frames especificados
for frame_id in range(inicio, fim):
    # Posicionar o vídeo no frame desejado
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)

    # Ler o frame
    ret, frame = cap.read()
    if not ret:
        break

    # Crop do frame
    cropped_frame = frame[color_crop_rows[0]:color_crop_rows[1], color_crop_cols[0]:color_crop_cols[1]]

    # Converter para tensor
    img_rgb = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2RGB)
    img_tensor = torchvision.transforms.ToTensor()(img_rgb)
    img_tensor = img_tensor.to(device)

    # Avaliação do modelo sem gradientes
    model.eval()
    with torch.no_grad():
        output = model([img_tensor])

    # Visualizar os resultados
    img = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    img = np.ascontiguousarray(img, dtype=np.uint8)

    # Extrair boxes, scores e labels
    boxes = output[0]['boxes'].cpu().numpy()
    scores = output[0]['scores'].cpu().numpy()
    labels = output[0]['labels'].cpu().numpy()

    # Desenhar as caixas delimitadoras
    for box, score, label in zip(boxes, scores, labels):
        if score > 0.5:  # Threshold de confiança
            xtl, ytl, xbr, ybr = box.astype(int)
            cv2.rectangle(img, (xtl, ytl), (xbr, ybr), (0, 255, 0), 2)
            label_name = list(label_map.keys())[list(label_map.values()).index(label)]
            cv2.putText(img, f"{label_name}: {score:.2f}", (xtl, ytl-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2)

    # Converter de volta para RGB para plotagem
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# Liberar o vídeo
cap.release()


Output hidden; open in https://colab.research.google.com to view.

In [21]:
# Salvar os pesos do modelo
torch.save(model.state_dict(), 'modelo_deteccao_olhos.pth')

# Salvar o modelo completo (incluindo a arquitetura)
torch.save(model, 'modelo_deteccao_olhos_completo.pth')
